In [ ]:
from dcicutils import ff_utils
from functions.notebook_functions import *
import pandas as pd
import boto3
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import json
import collections
import time
from gspread_formatting import *

In [ ]:
# Authenticating the credentials for google sheets
s3 = boto3.resource('s3')
obj = s3.Object('elasticbeanstalk-fourfront-webprod-system', 'DCICjupgoogle.json')
cont = obj.get()['Body'].read().decode()
key_dict = json.loads(cont)
SCOPES = 'https://www.googleapis.com/auth/spreadsheets'
creds = ServiceAccountCredentials.from_json_keyfile_dict(key_dict, SCOPES)
gc = gspread.authorize(creds)

In [ ]:
# Get all the grants from the Portal
my_key = get_key('default')
all_awards = ff_utils.search_metadata('/search/?type=Award', key=my_key)

#Get all the centers info from google management sheets
gs_management_sheet_id = ''
gs_management_workbook = gc.open_by_key(gs_management_sheet_id)
gs_management_worksheet = gs_management_workbook.sheet1


In [ ]:
centers_sheets_links = gs_management_worksheet.get_all_values()

In [ ]:
df = pd.DataFrame(centers_sheets_links[1:], columns=centers_sheets_links[0])

In [ ]:
col_names = df.columns.values

In [ ]:
print(col_names)

In [ ]:
#New Columns
#Link for Center's google sheet
"https://docs.google.com/spreadsheets/d/{}".format(google_sheet_id)
# Contact PI name
#email address in tobox format
'"{}, {}" <{}>'.format(last_name, first_name, email)


In [ ]:
gs_link_column = []
contact_pi_column = []
email_adress_column = []
x = 0
for center in centers_sheets_links:
    if x == 0:
        x = x + 1
        continue
    for award in all_awards:
        if 'Guttman' in center[0]:
             center[1] = center[1].split(',')[0]

        if center[1] in award['link_id']:   
            gs_id = center[2]
            contact_pi_fname = award['pi']["first_name"]
            contact_pi_lname = award['pi']['last_name']
            contact_pi_email = award['pi']['contact_email']
            gs_link = "https://docs.google.com/spreadsheets/d/{}".format(gs_id)
            contact_pi_name = contact_pi_fname + ' ' + contact_pi_lname
            email = '"{}, {}" <{}>'.format(contact_pi_lname,  contact_pi_fname, contact_pi_email)
            break
    gs_link_column.append(gs_link)
    contact_pi_column.append(contact_pi_name)
    email_adress_column.append(email)  

In [ ]:
print(len(email_adress_column))

In [ ]:
df['Center google sheet'] = gs_link_column

In [ ]:
df['Contact PI'] = contact_pi_column

In [ ]:
df['Email address'] = email_adress_column

In [ ]:
print(df.columns.values)

In [ ]:
col_labels = df.columns.values

In [ ]:
data = df.values.tolist()

In [ ]:
gs_write = []
row = 2
col = 1
for label in col_labels:
    gs_write.append(gspread.models.Cell(1,col, label))
    col = col + 1

col = 1
for item in data:
    for ele in item:
        gs_write.append(gspread.models.Cell(row,col,ele))
        col = col + 1
    row = row + 1
    col = 1

In [ ]:
#print(gs_write)

In [ ]:
gs_management_worksheet.update_cells(gs_write, value_input_option='USER_ENTERED')